In [1]:
import gymnasium as gym
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

In [2]:
env = gym.make('FrozenLake-v1', desc=generate_random_map(size=5), render_mode="human", is_slippery=False)
observation, info = env.reset()

: 